# Learning Journey 2

Start with introducing the scenario:
- Company wants to analyse support mail
- See company description in `company-scope.md`
- Incoming mails should be assigned to the correct service category and assigned the correct urgency and sentiment

**Goal**: Analyse every message and extract:
- `urgency`
- `sentiment`
- `categories`

## Install packages

In [1]:
!pip install -U "generative-ai-hub-sdk>=3.1" tqdm

Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 17.078 seconds


In [2]:
import time
import json
import os
from IPython.display import clear_output
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from enum import Enum
 
# Inline credentials
with open('config.json') as f:
    credCF = json.load(f)

# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "default" 
    }

    for key, value in env_vars.items():
        os.environ[key] = value

# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )

ai_core_client = create_ai_core_client(credCF)

## Imports and Settings

In [3]:
from typing import Literal, Type, Union, Dict, Any, List, Callable
import re, pathlib, json, time
from functools import partial

In [4]:
EXAMPLE_MESSAGE_IDX = 10

Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 0.625 seconds


## Load Data

In [5]:
HERE = pathlib.Path.cwd()

with (HERE / 'filtered_mails-hardest.jsonl').open() as stream:
    mails = [json.loads(line) for line in stream if line.strip()]

In [6]:
dev_set, test_set = mails[:int(len(mails)/2)], mails[int(len(mails)/2):]
test_set_small = test_set[:20]

In [7]:
categories = set()
urgency = set()
sentiment = set()
for mail in mails:
    categories = categories.union(set(mail['ground_truth']['categories']))
    urgency.add(mail['ground_truth']['urgency'])
    sentiment.add(mail['ground_truth']['sentiment'])

option_lists = {
    'urgency': ', '.join(f"`{entry}`" for entry in urgency),
    'sentiment': ', '.join(f"`{entry}`" for entry in sentiment),
    'categories': ', '.join(f"`{entry}`" for entry in categories),
}

In [8]:
option_lists

{'urgency': '`medium`, `high`, `low`',
 'sentiment': '`neutral`, `negative`, `positive`',
 'categories': '`emergency_repair_services`, `cleaning_services_scheduling`, `sustainability_and_environmental_practices`, `training_and_support_requests`, `quality_and_safety_concerns`, `specialized_cleaning_services`, `general_inquiries`, `customer_feedback_and_complaints`, `facility_management_issues`, `routine_maintenance_requests`'}

## Helper Functions

In [9]:
from gen_ai_hub.proxy import get_proxy_client


client = get_proxy_client()

In [10]:
from typing import Callable

from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_api_client_sdk.models.status import Status
import time
from IPython.display import clear_output


def spinner(check_callback: Callable, timeout: int = 300, check_every_n_seconds: int = 10):
    start = time.time()
    last_check = start
    while time.time() - start < timeout:
        now = time.time()
        if now - start > timeout:
            break
        if now - last_check > check_every_n_seconds:
            return_value = check_callback()
            if return_value:
                return return_value
        for char in '|/-\\':
            clear_output(wait=True)  # Clears the output to show a fresh update
            print(f'Waiting for the deployment to become ready... {char}')
            time.sleep(0.2)  # Adjust the speed as needed


def retrieve_or_deploy_orchestration(ai_core_client: AICoreV2Client,
                                     scenario_id: str = "orchestration",
                                     executable_id: str = "orchestration",
                                     config_suffix: str = "simple",
                                     start_timeout: int = 300):
    if not config_suffix:
        raise ValueError("Empty `config_suffix` not allowed")
    deployments = ai_core_client.deployment.query(
        scenario_id=scenario_id,
        executable_ids=[executable_id],
        status=Status.RUNNING
    )
    if deployments.count > 0:
        return sorted(deployments.resources, key=lambda x: x.start_time)[0]
    config_name = f"{config_suffix}-orchestration"
    configs = ai_core_client.configuration.query(
        scenario_id=scenario_id,
        executable_ids=[executable_id],
        search=config_name
    )
    if configs.count > 0:
        config = sorted(deployments.resources, key=lambda x: x.start_time)[0]
    else:
        config = ai_core_client.configuration.create(
            scenario_id=scenario_id,
            executable_id=executable_id,
            name=config_name,
        )
    deployment = ai_core_client.deployment.create(configuration_id=config.id)

    def check_ready():
        updated_deployment = ai_core_client.deployment.get(deployment.id)
        return None if updated_deployment.status != Status.RUNNING else updated_deployment
    
    return spinner(check_ready)

In [11]:
import pathlib
import yaml

from gen_ai_hub.proxy import get_proxy_client
from ai_api_client_sdk.models.status import Status

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.content_filter import AzureContentFilter
from gen_ai_hub.orchestration.service import OrchestrationService

client = get_proxy_client()
deployment = retrieve_or_deploy_orchestration(client.ai_core_client)
orchestration_service = OrchestrationService(api_url=deployment.deployment_url, proxy_client=client)

def send_request(prompt, _print=True, _model='meta--llama3-70b-instruct', **kwargs):
    config = OrchestrationConfig(
        llm=LLM(name=_model),
        template=Template(messages=[UserMessage(prompt)])
    )
    template_values = [TemplateValue(name=key, value=value) for key, value in kwargs.items()]
    answer = orchestration_service.run(config=config, template_values=template_values)
    result = answer.module_results.llm.choices[0].message.content
    if _print:
        formatted_prompt = answer.module_results.templating[0].content
        print(f"<-- PROMPT --->\n{formatted_prompt if _print else prompt}\n<--- RESPONSE --->\n{result}")   
    return result


## Sentiment & Urgency Simple

In [12]:
mail = dev_set[EXAMPLE_MESSAGE_IDX]

In [13]:
prompt_1 = """Giving the following message:
---
{{?input}}
---
Your task is to extract
- urgency
- sentiment
"""

f_1 = partial(send_request, prompt=prompt_1)

response = f_1(input=mail["message"])

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 17.156 seconds


## Sentiment & Urgency with Choices

In [14]:
prompt_2 = """Giving the following message:
---
{{?input}}
---
Your task is to extract:
- "urgency" as one of {urgency}
- "sentiment" as one of {sentiment}
"""

f_2 = partial(send_request, prompt=prompt_2, **option_lists)

response = f_2(input=mail["message"])

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

## Sentiment & Urgency with Choices as `json`

In [15]:
prompt_3 = """Giving the following message:
---
{{?input}}
---
Extract and return a json with the follwoing keys and values:
- "urgency" as one of {{?urgency}}
- "sentiment" as one of {{?sentiment}}
Your complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above.
"""
f_3 = partial(send_request, prompt=prompt_3, **option_lists)

response = f_3(input=mail["message"])

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

## Sentiment & Urgency with Choices as `json` Improved Formatting

In [16]:
prompt_4 = """Giving the following message:
---
{{?input}}
---
Extract and return a json with the follwoing keys and values:
- "urgency" as one of {{?urgency}}
- "sentiment" as one of {{?sentiment}}
Your complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces."""
f_4 = partial(send_request, prompt=prompt_4, **option_lists)

response = f_4(input=mail["message"])

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

## Category Simple

In [17]:
prompt_5 = """Giving the following message:
---
{{?input}}
---
Assign a list of matching support category to the message.
"""
f_5 = partial(send_request, prompt=prompt_5)

response = f_5(input=mail["message"])

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

## Categories form list

In [18]:
prompt_6 = """Giving the following message:
---
{{?input}}
---
Assign a list best matching support category tags to the message:
{{?categories}}
"""
f_6 = partial(send_request, prompt=prompt_6, **option_lists)

response = f_6(input=mail["message"])

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

## Category with  Descriptions and `json` Output

In [19]:
prompt_7 = """Giving the following message:
---
{{?input}}
---
Extract and return a json with the follwoing keys and values:
- "categories" list of the best matching support category tags from: {{?categories}}
Your complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.
"""
f_7 = partial(send_request, prompt=prompt_7, **option_lists)

response = f_7(input=mail["message"])

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

## Complete

In [20]:
prompt_8 = """Giving the following message:
---
{{?input}}
---
Extract and return a json with the follwoing keys and values:
- "urgency" as one of {{?urgency}}
- "sentiment" as one of {{?sentiment}}
- "categories" list of the best matching support category tags from: {{?categories}}
Your complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.
"""
f_8 = partial(send_request, prompt=prompt_8, **option_lists)

response = f_8(input=mail["message"])

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

# Evaluation

In [21]:
from tqdm.auto import tqdm
import time


class RateLimitedIterator:
    def __init__(self, iterable, max_iterations_per_minute):
        self._iterable = iter(iterable)
        self._max_iterations_per_minute = max_iterations_per_minute
        self._min_interval = 1.0 / (max_iterations_per_minute / 60.)
        self._last_yield_time = None

    def __iter__(self):
        return self

    def __next__(self):
        current_time = time.time()

        if self._last_yield_time is not None:
            elapsed_time = current_time - self._last_yield_time
            if elapsed_time < self._min_interval:
                time.sleep(self._min_interval - elapsed_time)

        self._last_yield_time = time.time()
        return next(self._iterable)


In [22]:
def evaluation(mail: Dict[str, str], extract_func: Callable, _print=True, **kwargs):
    response = extract_func(input=mail["message"], _print=_print, **kwargs)
    result = {
        "is_valid_json": False,
        "correct_categories": False,
        "correct_sentiment": False,
        "correct_urgency": False,
    }
    try:
        pred = json.loads(response)
    except json.JSONDecodeError:
        result["is_valid_json"] = False
    else:
        result["is_valid_json"] = True
        result["correct_categories"] = 1 - (len(set(mail["ground_truth"]["categories"]) ^ set(pred["categories"])) / len(categories))
        result["correct_sentiment"] = pred["sentiment"] == mail["ground_truth"]["sentiment"]
        result["correct_urgency"] = pred["urgency"] == mail["ground_truth"]["urgency"]
    return result

In [23]:
evaluation(mail, f_8)

<-- PROMPT --->
Giving the following message:
---
Subject: Urgent HVAC System Repair Needed

Dear Support Team,

I hope this message finds you well. My name is [Sender], and I am reaching out to you from [Residential Complex Name], where I have been residing for the past few years. I have always appreciated the meticulous care and attention your team provides in maintaining our facilities.

However, I am currently facing a pressing issue with the HVAC system in my apartment. Over the past few days, the system has been malfunctioning, resulting in inconsistent temperatures and, at times, complete shutdowns. Given the current weather conditions, this has become quite unbearable and is affecting my daily routine significantly.

I have attempted to troubleshoot the problem by resetting the system and checking the thermostat settings, but these efforts have not yielded any improvement. The situation seems to be beyond my control and requires professional intervention.

I kindly request that

{'is_valid_json': True,
 'correct_categories': 0.8,
 'correct_sentiment': False,
 'correct_urgency': True}

In [24]:
from tqdm.auto import tqdm

def transpose_list_of_dicts(list_of_dicts):
    keys = list_of_dicts[0].keys()
    transposed_dict = {key: [] for key in keys}
    for d in list_of_dicts:
        for key, value in d.items():
            transposed_dict[key].append(value)
    return transposed_dict

def evalulation_full_dataset(dataset, func, rate_limit=100, _print=False, **kwargs):
    results = [evaluation(mail, func, _print=_print, **kwargs) for mail in tqdm(RateLimitedIterator(dataset, rate_limit), total=len(dataset))]
    results = transpose_list_of_dicts(results)
    n = len(dataset)
    for k, v in results.items():
        results[k] = sum(v) / len(dataset)
    return results


def pretty_print_table(data):
    # Get all row names (outer dict keys)
    row_names = list(data.keys())

    # Get all column names (inner dict keys)
    if row_names:
        column_names = list(data[row_names[0]].keys())
    else:
        column_names = []

    # Calculate column widths
    column_widths = [max(len(str(column_name)), max(len(f"{data[row][column_name]:.2f}") for row in row_names)) for column_name in column_names]
    row_name_width = max(len(str(row_name)) for row_name in row_names)

    # Print header
    header = f"{'':>{row_name_width}} " + " ".join([f"{column_name:>{width}}" for column_name, width in zip(column_names, column_widths)])
    print(header)
    print("=" * len(header))

    # Print rows
    for row_name in row_names:
        row = f"{row_name:>{row_name_width}} " + " ".join([f"{data[row_name][column_name]:>{width}.1%}" for column_name, width in zip(column_names, column_widths)])
        print(row)

overall_result = {}

In [25]:
overall_result["basic--llama3-70b"] = evalulation_full_dataset(test_set_small, f_8)
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                  is_valid_json correct_categories correct_sentiment correct_urgency
basic--llama3-70b        100.0%              86.0%             40.0%           65.0%


Rerunning the cell above shows that the output depends a lot on the given input example

## Few-Shot Example

In [26]:
prompt_10 = """Your task is to extract and categorize messages. Here are some example:
---
{{?few_shot_examples}}
---
Use the examples when extract and categorize the following message:
---
{{?input}}
---
Extract and return a json with the follwoing keys and values:
- "urgency" as one of {{?urgency}}
- "sentiment" as one of {{?sentiment}}
- "categories" list of the best matching support category tags from: {{?categories}}
Your complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.
"""

import random
random.seed(42)

k = 3
examples = random.sample(dev_set, k)

example_template = """<example>
{example_input}

## Output

{example_output}
</example>"""

examples = '\n---\n'.join([example_template.format(example_input=example["message"], example_output=json.dumps(example["ground_truth"])) for example in examples])


f_10 = partial(send_request, prompt=prompt_10, few_shot_examples=examples, **option_lists)

response = f_10(input=mail["message"])

<-- PROMPT --->
Your task is to extract and categorize messages. Here are some example:
---
<example>
Subject: Inquiry About Training Programs for In-House Maintenance Team

Hi Support Team,

I hope this message finds you well. My name is Alex, and I’ve been a satisfied customer of Facility Solutions Company for the past year. I’m reaching out today with a request for more information regarding your training programs.

As a parent who values evidence-based practices, I’m particularly interested in ensuring that our in-house maintenance team is equipped with the latest knowledge and skills. We’ve been managing our residential complex with a focus on sustainability and efficiency, and I believe that proper training is crucial for maintaining these standards.

Could you provide me with details about the training programs you offer? Specifically, I’m looking for information on the curriculum, duration, and any certifications that might be included. Additionally, I’d like to know if there a

In [27]:
overall_result["few_shot--llama3-70b"] = evalulation_full_dataset(test_set_small, f_10)
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                     is_valid_json correct_categories correct_sentiment correct_urgency
   basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%


## Use Metaprompting

In [28]:
example_template_metaprompt = """<example>
{example_input}

## Output
{key}={example_output}
</example>"""

prompt_get_guide = """Here are some example:
---
{{?examples}}
---
Use the examples above to come up with a guide on how to distinguish between {{?options}} {{?key}}.
Use the following format:
```
### **<category 1>**
- <instruction 1>
- <instruction 2>
- <instruction 3>
### **<category 2>**
- <instruction 1>
- <instruction 2>
- <instruction 3>
...
```
When creating the guide:
- make it step-by-step instructions
- Consider than some labels in the examples might be in correct
- Avoid including explicit information from the examples in the guide
The guide has to cover: {{?options}}
"""

guides = {}

for i, key in enumerate(["categories", "urgency", "sentiment"]):
    options = option_lists[key]
    selected_examples_txt_metaprompt = '\n---\n'.join([example_template_metaprompt.format(example_input=example["message"], key=key, example_output=example["ground_truth"][key]) for example in dev_set])
    guides[f"guide_{key}"] = send_request(prompt=prompt_get_guide, examples=selected_examples_txt_metaprompt, key=key, options=options, _print=False, _model='gpt-4o')

In [29]:
print(guides['guide_urgency'])

### **High Urgency**
- **Determine if the issue is causing a severe immediate impact:**
  - Assess if the situation is critical and negatively affecting day-to-day operations or safety.
  - Examples include HVAC breakdowns causing discomfort, emergency repairs needed, or immediate safety hazards.
- **Check for a sense of urgency in the language:**
  - Look for words like "urgent," "immediate," "critical," or "emergency."
  - The sender is often insistent that the issue must be resolved as soon as possible.
- **Review attempts for previous resolutions:**
  - Note if the sender has already tried resolving the issue internally or through initial contact without success and is now seeking urgent intervention.

### **Medium Urgency**
- **Assess the impact on functionality or comfort:**
  - Determine if the issue affects the normal functioning or comfort level but is not critically urgent. 
  - Examples include routine maintenance requests where the system is not fully broken but underperfor

In [30]:
prompt_12 = """Your task is to classify messages.
This is an explanation of `urgency` labels:
---
{{?guide_urgency}}
---
This is an explanation of `sentiment` labels:
---
{{?guide_sentiment}}
---
This is an explanation of `support` categories:
---
{{?guide_categories}}
---
Giving the following message:
---
{{?input}}
---
Extract and return a json with the follwoing keys and values:
- "urgency" as one of {{?urgency}}
- "sentiment" as one of {{?sentiment}}
- "categories" list of the best matching support category tags from: {{?categories}}
Your complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.
"""
f_12 = partial(send_request, prompt=prompt_12, **option_lists, **guides)
response = f_12(input=mail["message"])

<-- PROMPT --->
Your task is to classify messages.
This is an explanation of `urgency` labels:
---
### **High Urgency**
- **Determine if the issue is causing a severe immediate impact:**
  - Assess if the situation is critical and negatively affecting day-to-day operations or safety.
  - Examples include HVAC breakdowns causing discomfort, emergency repairs needed, or immediate safety hazards.
- **Check for a sense of urgency in the language:**
  - Look for words like "urgent," "immediate," "critical," or "emergency."
  - The sender is often insistent that the issue must be resolved as soon as possible.
- **Review attempts for previous resolutions:**
  - Note if the sender has already tried resolving the issue internally or through initial contact without success and is now seeking urgent intervention.

### **Medium Urgency**
- **Assess the impact on functionality or comfort:**
  - Determine if the issue affects the normal functioning or comfort level but is not critically urgent. 
  -

In [31]:
overall_result["metaprompting--llama3-70b"] = evalulation_full_dataset(test_set_small, f_12)
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                          is_valid_json correct_categories correct_sentiment correct_urgency
        basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
     few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%


## Combine Few-Shot with Metaprompting

In [32]:
prompt_13 = """Your task is to classify messages.
Here are some examples:
---
{{?few_shot_examples}}
---
This is an explanation of `urgency` labels:
---
{{?guide_urgency}}
---
This is an explanation of `sentiment` labels:
---
{{?guide_sentiment}}
---
This is an explanation of `support` categories:
---
{{?guide_categories}}
---
Giving the following message:
---
{{?input}}
---
extract and return a json with the follwoing keys and values:
- "urgency" as one of {{?urgency}}
- "sentiment" as one of {{?sentiment}}
- "categories" list of the best matching support category tags from: {{?categories}}
Your complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.
"""

f_13 = partial(send_request, prompt=prompt_13, **option_lists, few_shot_examples=examples, **guides)

response = f_13(input=mail["message"])

<-- PROMPT --->
Your task is to classify messages.
Here are some examples:
---
<example>
Subject: Inquiry About Training Programs for In-House Maintenance Team

Hi Support Team,

I hope this message finds you well. My name is Alex, and I’ve been a satisfied customer of Facility Solutions Company for the past year. I’m reaching out today with a request for more information regarding your training programs.

As a parent who values evidence-based practices, I’m particularly interested in ensuring that our in-house maintenance team is equipped with the latest knowledge and skills. We’ve been managing our residential complex with a focus on sustainability and efficiency, and I believe that proper training is crucial for maintaining these standards.

Could you provide me with details about the training programs you offer? Specifically, I’m looking for information on the curriculum, duration, and any certifications that might be included. Additionally, I’d like to know if there are any upcomi

In [33]:
overall_result["metaprompting_and_few_shot--llama3-70b"] = evalulation_full_dataset(test_set_small, f_13)
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                       is_valid_json correct_categories correct_sentiment correct_urgency
                     basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                  few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
             metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%


## Use `mistralai--mixtral-8x7b-instruct-v01`

As the cheapest open source, SAP hosted model available on generative AI hub

In [34]:
overall_result["basic--mixtral-8x7b"] = evalulation_full_dataset(test_set_small, f_8, _model='mistralai--mixtral-8x7b-instruct-v01')
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                       is_valid_json correct_categories correct_sentiment correct_urgency
                     basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                  few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
             metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%
                   basic--mixtral-8x7b        100.0%              87.0%             45.0%           50.0%


In [35]:
overall_result["metaprompting_and_few_shot--mixtral-8x7b"] = evalulation_full_dataset(test_set_small, f_13, _model='mistralai--mixtral-8x7b-instruct-v01')
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                         is_valid_json correct_categories correct_sentiment correct_urgency
                       basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                    few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
               metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
  metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%
                     basic--mixtral-8x7b        100.0%              87.0%             45.0%           50.0%
metaprompting_and_few_shot--mixtral-8x7b        100.0%              92.5%             70.0%           75.0%


## Use `gpt-4o`

As the best propriatary OpenAI available on generative AI hub

In [36]:
overall_result["basic--gpt4o"] = evalulation_full_dataset(test_set_small, f_8, _model='gpt-4o')
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                         is_valid_json correct_categories correct_sentiment correct_urgency
                       basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                    few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
               metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
  metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%
                     basic--mixtral-8x7b        100.0%              87.0%             45.0%           50.0%
metaprompting_and_few_shot--mixtral-8x7b        100.0%              92.5%             70.0%           75.0%
                            basic--gpt4o        100.0%              90.0%             35.0%           50.0%


In [37]:
overall_result["metaprompting_and_few_shot--gpt4o"] = evalulation_full_dataset(test_set_small, f_13, _model='gpt-4o')
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                         is_valid_json correct_categories correct_sentiment correct_urgency
                       basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                    few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
               metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
  metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%
                     basic--mixtral-8x7b        100.0%              87.0%             45.0%           50.0%
metaprompting_and_few_shot--mixtral-8x7b        100.0%              92.5%             70.0%           75.0%
                            basic--gpt4o        100.0%              90.0%             35.0%           50.0%
       metaprompting_and_few_shot--gpt4o        100.0%              93.0%             70.0%           85.0%


## Using `gemini-1.5-flash`

As the cheapest and fastest Google model available on generative AI hub

In [38]:
overall_result["basic--gemini-1.5-flash"] = evalulation_full_dataset(test_set_small, f_8, _model='gemini-1.5-flash')
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                         is_valid_json correct_categories correct_sentiment correct_urgency
                       basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                    few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
               metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
  metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%
                     basic--mixtral-8x7b        100.0%              87.0%             45.0%           50.0%
metaprompting_and_few_shot--mixtral-8x7b        100.0%              92.5%             70.0%           75.0%
                            basic--gpt4o        100.0%              90.0%             35.0%           50.0%
       metaprompting_and_few_shot--gpt4o        100.0%              93.0%             70.0%           85.0%
                 basic--gemi

In [39]:
overall_result["metaprompting_and_few_shot--gemini-1.5-flash"] = evalulation_full_dataset(test_set_small, f_13, _model='gemini-1.5-flash')
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                             is_valid_json correct_categories correct_sentiment correct_urgency
                           basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                        few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
                   metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
      metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%
                         basic--mixtral-8x7b        100.0%              87.0%             45.0%           50.0%
    metaprompting_and_few_shot--mixtral-8x7b        100.0%              92.5%             70.0%           75.0%
                                basic--gpt4o        100.0%              90.0%             35.0%           50.0%
           metaprompting_and_few_shot--gpt4o        100.0%              93.0%             70.0%         

## Use `gemini-1.5-pro`

Best Google model available on generative AI hub

In [40]:
overall_result["basic--gemini-1.5-pro"] = evalulation_full_dataset(test_set_small, f_8, _model='gemini-1.5-pro')
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                             is_valid_json correct_categories correct_sentiment correct_urgency
                           basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                        few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
                   metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
      metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%
                         basic--mixtral-8x7b        100.0%              87.0%             45.0%           50.0%
    metaprompting_and_few_shot--mixtral-8x7b        100.0%              92.5%             70.0%           75.0%
                                basic--gpt4o        100.0%              90.0%             35.0%           50.0%
           metaprompting_and_few_shot--gpt4o        100.0%              93.0%             70.0%         

Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 750.734 seconds


In [41]:
overall_result["metaprompting_and_few_shot--gemini-1.5-pro"] = evalulation_full_dataset(test_set_small, f_13, _model='gemini-1.5-pro')
pretty_print_table(overall_result)

  0%|          | 0/20 [00:00<?, ?it/s]

                                             is_valid_json correct_categories correct_sentiment correct_urgency
                           basic--llama3-70b        100.0%              86.0%             40.0%           65.0%
                        few_shot--llama3-70b        100.0%              86.0%             60.0%           80.0%
                   metaprompting--llama3-70b        100.0%              88.0%             40.0%           90.0%
      metaprompting_and_few_shot--llama3-70b        100.0%              88.5%             70.0%           85.0%
                         basic--mixtral-8x7b        100.0%              87.0%             45.0%           50.0%
    metaprompting_and_few_shot--mixtral-8x7b        100.0%              92.5%             70.0%           75.0%
                                basic--gpt4o        100.0%              90.0%             35.0%           50.0%
           metaprompting_and_few_shot--gpt4o        100.0%              93.0%             70.0%         

Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\I746414\AppData\Local\anaconda3\Lib\site-packages\tornado\ioloop.py:685] created at C:\Users\I746414\AppData\Local\anaconda3\Lib\asyncio\tasks.py:670> took 64.219 seconds
